In [1]:
#abc

In [2]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d awsaf49/brats20-dataset-training-validation

100% 4.15G/4.16G [00:34<00:00, 134MB/s]
100% 4.16G/4.16G [00:34<00:00, 128MB/s]


In [ ]:
! unzip brats20-dataset-training-validation.zip

In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import tensorflow as tf
import cv2
import os
import random
import numpy as np
import glob

from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tqdm
import matplotlib.pyplot as plt

In [7]:
sorted(os.listdir('/content/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'))[-3]

'BraTS20_Training_369'

In [10]:
os.listdir('/content/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355')

['BraTS20_Training_355_flair.nii',
 'BraTS20_Training_355_seg.nii',
 'BraTS20_Training_355_t2.nii',
 'BraTS20_Training_355_t1.nii',
 'BraTS20_Training_355_t1ce.nii']

In [9]:
directory_path = '/content/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_355'
files = os.listdir(directory_path)

# Find the file you want to rename
for filename in files:
    if filename == "W39_1998.09.19_Segm.nii":
        # Rename the file
        os.rename(os.path.join(directory_path, filename), os.path.join(directory_path, "BraTS20_Training_355_seg.nii"))

In [ ]:
import os
import numpy as np
import nibabel as nib

class CustomDataGenerator:
    def __init__(self, data_path, batch_size):
        self.data_path = data_path
        self.batch_size = batch_size
        self.image_names = []
        self.mask_names = []
        for dirpath, dirnames, filenames in os.walk(self.data_path):
            for filename in filenames:
                if filename.endswith(".nii"):
                    if "mask" in filename:
                        self.mask_names.append(os.path.join(dirpath, filename))
                    else:
                        self.image_names.append(os.path.join(dirpath, filename))
        self.n_samples = len(self.image_names) // 2

    def __len__(self):
        return int(np.ceil(self.n_samples / float(self.batch_size)))

    def __getitem__(self, index):
        indexes = range(index*self.batch_size, min((index+1)*self.batch_size, self.n_samples))
        batch_image_names = [self.image_names[i] for i in indexes for _ in range(2)]
        batch_mask_names = [self.mask_names[i] for i in indexes]
        X = []
        Y = []
        for i in range(len(batch_image_names)):
            image_data = nib.load(batch_image_names[i]).get_fdata()
            X.append(image_data)
            if (i+1) % 2 == 0:
                mask_data = nib.load(batch_mask_names[i//2]).get_fdata()
                Y.append(mask_data)
        X = np.stack(X, axis=-1)
        Y = np.stack(Y, axis=-1)
        return X, Y
